## Building out the Neural Network

From [Andrej Karpathy: The spelled-out intro to neural networks and backpropagation: building micrograd ](https://youtu.be/VMj-3S1tku0?t=5978)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%matplotlib inline

In [2]:
from graphviz import Digraph

# PyTorch Demonstration
duplicates the features of Micrograd

In [3]:
import torch

In [4]:
# pytorch has the goal of efficiency by producing operations in parallel by working on 
# tensors instead of scaler values 

# tensors are N dimensional arrays of scalers
t = torch.Tensor([[1,2,3], [4,5,6]])
print(t.shape)
t

torch.Size([2, 3])


tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [5]:
# by default pytorch as float32 precision
t.dtype

torch.float32

In [6]:
# pytorch requires tensors instead of scalers, here tensors are 1D arrays
# by default pytorch has dtype float32, cast as double for dtype float64 to mirror python
# pytorch assumes leaf nodes do not require gradients, requires_grad = True calculates gradients on leaves
x1 = torch.Tensor([2.0]).double(); x1.requires_grad = True
x2 = torch.Tensor([0.0]).double(); x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double(); w1.requires_grad = True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad = True

b = torch.Tensor([6.8813735870195432]).double(); x1.requires_grad = True

In [7]:
x1.dtype

torch.float64

In [8]:
n = x1*w1 + x2*w2 + b

In [9]:
o = torch.tanh(n)

In [10]:
print(o.data.item())
o.backward()

0.7071066904050358


In [11]:
print('----')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

----
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [12]:
print(o.item())
o

0.7071066904050358


tensor([0.7071], dtype=torch.float64, grad_fn=<TanhBackward0>)

# In Micrograd

In [13]:
def trace(root):
    # builds a set of all nodes and edges in a graph -- from a Value Object
    
    nodes, edges = set(), set()
    
    def build(v):
    # build reaches up out of scope to access nodes and edges... 
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                # recursive call to elaborate all of the child nodes 
                build(child)
    build(root)
    return nodes, edges


In [14]:
def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        
        # for any value in the graph, create a rectangular ('record') node for it
        # each node displays label, data, and gradient at that leaf
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape = 'record')
        
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            
            # and connect this node to it
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [15]:
class Value:
    # _children and _op at init requires values, used for history
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        # gradient at each leaf/Value
        self.grad = 0 # initially set to 0 with the assumption that the leaf/Value
                      # does not effect the overall outcome (Loss Function)
            
        # backward as a attribute of the Value object
        # stores the gradient calculation at a given value (node)
        # by default None, depends on the operation and the children nodes
        self._backward = lambda: None
            
        # to capture the previous values and operations
        self._prev = set(_children)
        self._op = _op  # operation performed at the node
        self.label = label # name of the node
        
    def __repr__(self):
        # necessary to print out something readable -- otherwise shows
        # the function and location in memory
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        # other must be a second Value object
        # pass children values to capture history
        out = Value(self.data + other.data, (self, other), '+')
        
        # closure to manage the backpropagation local to the operation
        def _backward():
            # addition propagates the gradient forward 
#            self.grad = 1.0 * out.grad
#            other.grad = 1.0 * out.grad
            # += is required if a Value object is used more than once, otherwise
            # the operation will over-write itself
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        # an addition node has this function as the action for backpropagation
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*') 
        
        def _backward():
#            self.grad = out.grad * other.data
#            other.grad = out.grad * self.data
            # += is required if a Value object is used more than once, otherwise
            # the operation will over-write itself
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward    
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supports ints and floats"
        out = Value(self.data ** other, (self,), f"**{other}")
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    def __rmul__(self, other): # use case other * self
        return self * other
    
    def __truediv__(self, other):  # for self/other
        return self * other**-1
    
    def __neg__(self): # - self
        return self * -1
    
    def __sub__(self, other): # self - other
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad = (1 - t**2) *out.grad # local gradient chained to the prior gradient
            
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
            
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [16]:
class Neuron:
    
    def __init__(self, nin):
        # takes a number of inputs on initialization
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        
    def __call__(self, x):
        # w * x + b <-- forward pass of neuron in the network
        
        # create the activation of the neuron
        # python sum takes a second argument, where to start the sum -- self.b
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)

        # return the layer of the neuron passed through an regularization function
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
        

In [17]:
# x is input data, the weights and biases will be trained from the network... 
x = [2.0, 3.0]
n = Neuron(2)

In [18]:
# randomized values
n(x)

Value(data=-0.9863700181144841)

### Layer of Neurons

In [19]:
# set of neurons evaluated independently

class Layer:
    # defined by number of inputs and outputs 
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    # less concise code with the same return
#         params = []
#         for neuron in self.neurons:
#             ps = neuron.parameters()
#             params.extend(ps)
#         return params

In [20]:
x = [2.0, 3.0]
n = Layer(2,3)
n(x)

[Value(data=-0.9970934107637096),
 Value(data=0.9895868071156434),
 Value(data=0.9989143021311471)]

## Multi-Layer Perceptron
multiple layers of neurons

In [21]:
class MLP:
    
    #nouts defines the size of layers for MLP
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        # x is 
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [22]:
# forward pass of MLP
x = [2.0, 3.0, -1.0]
n = MLP(3, [4,4,1]) # 3 inputs passing through 2 layers of 4 and a single output unit
n(x)

Value(data=0.8049700608291973)

In [23]:
# draw_dot(n(x))

In [24]:
# example; binary classifer nn; given 4 samples, derive 4 values 
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [25]:
# initial, untrained outputs 
ypred = [n(x) for x in xs]
ypred

[Value(data=0.8049700608291973),
 Value(data=0.37417956209333747),
 Value(data=-0.1499491610068778),
 Value(data=0.755680931853579)]

Tune the nn by observing Loss

In [26]:
# mean square error loss
loss = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]

In [27]:
loss

[Value(data=0.038036677172967),
 Value(data=1.8883694688750368),
 Value(data=0.7225864288729109),
 Value(data=0.059691807059935514)]

In [28]:
# minimize this number...
loss = sum(loss)
loss

Value(data=2.7086843819808504)

In [29]:
loss.backward()

In [30]:
n.layers[0].neurons[0].w[0].grad

2.335181345066107

In [31]:
n.layers[0].neurons[0].w[0].data

0.16150411701751666

In [32]:
# diagram of the data, weights and gradients within the nn
# xs is immutable - this is the data given for the problem
# the w values are the ones to be adjusted through the gradients
# paramters are the methods on the neurons, layers and MLP that return weights and biases
# of the neural net

# draw_dot(loss)

In [33]:
# n is the MLP
n.parameters() # 41 weights and biases in the nn

[Value(data=0.16150411701751666),
 Value(data=0.5237885046510045),
 Value(data=0.645650248367226),
 Value(data=-0.04955044632824479),
 Value(data=-0.20051054243668975),
 Value(data=-0.3981010019958944),
 Value(data=-0.708260022944112),
 Value(data=0.7560073937041072),
 Value(data=-0.8264845568076951),
 Value(data=-0.34467673281587663),
 Value(data=-0.8403862327005653),
 Value(data=0.020382741149430172),
 Value(data=0.6823684981447007),
 Value(data=0.9508410941237921),
 Value(data=-0.4729534863257183),
 Value(data=-0.5552236811559363),
 Value(data=-0.4207480300090467),
 Value(data=0.43505322237413613),
 Value(data=0.6912233912910715),
 Value(data=0.8528368428619546),
 Value(data=-0.6795429472684902),
 Value(data=-0.33008442519508985),
 Value(data=-0.9772171672259984),
 Value(data=-0.6344815688829812),
 Value(data=-0.79310116341313),
 Value(data=0.09088596448039787),
 Value(data=0.7556857420389691),
 Value(data=-0.39684057698508357),
 Value(data=0.6974886617520921),
 Value(data=-0.622181

### Forward pass

In [34]:
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=2.7086843819808504)

### Backward pass

In [35]:
loss.backward()

### Update

In [36]:
# all of the gradients express a vector that increases the loss --> use a negative value

for p in n.parameters():
    p.data += -0.01 * p.grad

In [37]:
n.layers[0].neurons[0].w[0].data

0.1148004901161945

In [38]:
loss

Value(data=2.7086843819808504)

In [39]:
ys

[1.0, -1.0, -1.0, 1.0]

In [40]:
ypred

[Value(data=0.8049700608291973),
 Value(data=0.37417956209333747),
 Value(data=-0.1499491610068778),
 Value(data=0.755680931853579)]

In [41]:
n.parameters()

[Value(data=0.1148004901161945),
 Value(data=0.5419171759480438),
 Value(data=0.6355287424947764),
 Value(data=-0.06388225721582987),
 Value(data=-0.3315647651121827),
 Value(data=-0.370912363645728),
 Value(data=-0.754732440723496),
 Value(data=0.6944800526126003),
 Value(data=-0.828025240524645),
 Value(data=-0.3459553119665867),
 Value(data=-0.8373622773342956),
 Value(data=0.01929708151026685),
 Value(data=0.48315574069538225),
 Value(data=0.9716414796468695),
 Value(data=-0.5427051573536429),
 Value(data=-0.65430015269308),
 Value(data=-0.41643723736247085),
 Value(data=0.43412365626243293),
 Value(data=0.6794938555644163),
 Value(data=0.8535082147101573),
 Value(data=-0.6693029916459798),
 Value(data=-0.30494300253135476),
 Value(data=-0.9788046432986597),
 Value(data=-0.6916466990313559),
 Value(data=-0.7791062708846048),
 Value(data=0.14926062718721234),
 Value(data=0.7845111432973997),
 Value(data=-0.40305831893250305),
 Value(data=0.6416839796937411),
 Value(data=-0.609343999

### Automating the cycle

In [45]:
for k in range(20):
    
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    
    # zero out the previous gradients
    for p in n.parameters():
        p.grad = 0.0
    
    # backward pass
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
        
    print(k, loss.data)

0 0.010634399512334164
1 0.010381458410525119
2 0.010140150017367431
3 0.009909693189271238
4 0.009689375034491271
5 0.009478543628413988
6 0.009276601641514882
7 0.00908300074939758
8 0.008897236715241636
9 0.008718845052210872
10 0.008547397187614356
11 0.0083824970624293
12 0.008223778109647086
13 0.00807090056313556
14 0.00792354905562292
15 0.007781430470225714
16 0.007644272014858601
17 0.007511819493027149
18 0.007383835748042889
19 0.007260099260717371


In [46]:
ypred

[Value(data=0.9657586999390567),
 Value(data=-0.9727198956728598),
 Value(data=-0.9511911693408983),
 Value(data=0.9455837653962965)]